In [ ]:
!ls

app.py	chrome_reviews.csv  cv-transform.pkl  knn_model.pkl  sample_data


Import Library

In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install -q streamlit_ace

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
import streamlit as st
import tqdm
import pickle

In [ ]:
stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
import re
def decontracted(text):
  text = re.sub(r"won't","will not",text)
  text = re.sub(r"can't","can not",text)
  text = re.sub(r"\'s","is",text)
  text = re.sub(r"\'re","are",text)
  text = re.sub(r"\'not","not",text)
  text = re.sub(r"\'ve","have",text)
  text = re.sub(r"\'d","would",text)
  text = re.sub(r"\'ll","will",text)
  text = re.sub(r"\'m","am",text)
  return text



In [ ]:
data = pd.read_csv('chrome_reviews.csv')
data.head()

ID                                         Review URL  \
0  3886  https://play.google.com/store/apps/details?id=...   
1  3887  https://play.google.com/store/apps/details?id=...   
2  3888  https://play.google.com/store/apps/details?id=...   
3  3889  https://play.google.com/store/apps/details?id=...   
4  3890  https://play.google.com/store/apps/details?id=...   

                                             Text  Star  Thumbs Up  \
0                      This is very helpfull aap.     5          0   
1                                            Good     3          2   
2  Not able to update. Neither able to uninstall.     1          0   
3                                        Nice app     4          0   
4                               Many unwanted ads     1          0   

          User Name Developer Reply        Version Review Date  \
0  INDIAN Knowledge             NaN  83.0.4103.106  2020-12-19   
1  Ijeoma Happiness             NaN  85.0.4183.127  2020-12-19   
2  Priti D BtCFs-29             NaN  85.0.4183.127  2020-12-19   
3        Ajeet Raja             NaN  77.0.3865.116  2020-12-19   
4           Rams Mp             NaN   87.0.4280.66  2020-12-19   

               App ID  
0  com.android.chrome  
1  com.android.chrome  
2  com.android.chrome  
3  com.android.chrome  
4  com.android.chrome

In [ ]:
data = data.drop(['Developer Reply'],axis=1)
data.head()

ID                                         Review URL  \
0  3886  https://play.google.com/store/apps/details?id=...   
1  3887  https://play.google.com/store/apps/details?id=...   
2  3888  https://play.google.com/store/apps/details?id=...   
3  3889  https://play.google.com/store/apps/details?id=...   
4  3890  https://play.google.com/store/apps/details?id=...   

                                             Text  Star  Thumbs Up  \
0                      This is very helpfull aap.     5          0   
1                                            Good     3          2   
2  Not able to update. Neither able to uninstall.     1          0   
3                                        Nice app     4          0   
4                               Many unwanted ads     1          0   

          User Name        Version Review Date              App ID  
0  INDIAN Knowledge  83.0.4103.106  2020-12-19  com.android.chrome  
1  Ijeoma Happiness  85.0.4183.127  2020-12-19  com.android.chrome  
2  Priti D BtCFs-29  85.0.4183.127  2020-12-19  com.android.chrome  
3        Ajeet Raja  77.0.3865.116  2020-12-19  com.android.chrome  
4           Rams Mp   87.0.4280.66  2020-12-19  com.android.chrome

In [ ]:
data.isnull().sum()

ID              0
Review URL      0
Text            1
Star            0
Thumbs Up       0
User Name       0
Version        85
Review Date     0
App ID          0
dtype: int64

In [ ]:
data.dropna(inplace = True)

In [ ]:
data.isnull().sum()

ID             0
Review URL     0
Text           0
Star           0
Thumbs Up      0
User Name      0
Version        0
Review Date    0
App ID         0
dtype: int64

In [ ]:
def rating(s):
  if(s>=3):
    return 1
  else:
    return 0
a = data['Star']
b = a.map(rating)
data['Star'] = b
print("number of data points in our data",data.shape)
data.head()

number of data points in our data (7118, 9)


ID                                         Review URL  \
0  3886  https://play.google.com/store/apps/details?id=...   
1  3887  https://play.google.com/store/apps/details?id=...   
2  3888  https://play.google.com/store/apps/details?id=...   
3  3889  https://play.google.com/store/apps/details?id=...   
4  3890  https://play.google.com/store/apps/details?id=...   

                                             Text  Star  Thumbs Up  \
0                      This is very helpfull aap.     1          0   
1                                            Good     1          2   
2  Not able to update. Neither able to uninstall.     0          0   
3                                        Nice app     1          0   
4                               Many unwanted ads     0          0   

          User Name        Version Review Date              App ID  
0  INDIAN Knowledge  83.0.4103.106  2020-12-19  com.android.chrome  
1  Ijeoma Happiness  85.0.4183.127  2020-12-19  com.android.chrome  
2  Priti D BtCFs-29  85.0.4183.127  2020-12-19  com.android.chrome  
3        Ajeet Raja  77.0.3865.116  2020-12-19  com.android.chrome  
4           Rams Mp   87.0.4280.66  2020-12-19  com.android.chrome

In [ ]:
pickle_in = open('knn_model.pkl',"rb")
classifier = pickle.load(pickle_in)


In [ ]:
'''import streamlit as st
import tqdm
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

pickle_in = open('Navie_model.pkl','rb')
classifier = pickle.load(pickle_in)

pickle_cv = open('cv-transform.pkl','rb')
cv = pickle.load(pickle_cv)
import re
def decontracted(text):

  text = re.sub(r"won't","will not",text)
  text = re.sub(r"can't","can not",text)
  text = re.sub(r"\'s","is",text)
  text = re.sub(r"\'re","are",text)
  text = re.sub(r"\'not","not",text)
  text = re.sub(r"\'ve","have",text)
  text = re.sub(r"\'d","would",text)
  text = re.sub(r"\'ll","will",text)
  text = re.sub(r"\'m","am",text)
  return text

def prediction(text):

  preprocessed_query = []
  text = decontracted(text)
  text = re.sub("\S*\d\S*","",text).strip() # remove words with numbers in python
  text = re.sub('[^A-Za-z0-9]+',' ',text)#remove special characters
  text = ' '.join(e.lower() for e in text.split() if e.lower() not in stopwords)
  preprocessed_query.append(text.strip())
  vect = cv.transform(preprocessed_query).toarray()
  print(vect)
  predictions = classifier.predict(vect)
  return predictions

ratings = prediction('this product is bad bad')
print(ratings)'''

'import streamlit as st\nimport tqdm\nimport warnings\nwarnings.filterwarnings(\'ignore\')\nimport pickle\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nstopwords = set([\'br\', \'the\', \'i\', \'me\', \'my\', \'myself\', \'we\', \'our\', \'ours\', \'ourselves\', \'you\', "you\'re", "you\'ve",            "you\'ll", "you\'d", \'your\', \'yours\', \'yourself\', \'yourselves\', \'he\', \'him\', \'his\', \'himself\',             \'she\', "she\'s", \'her\', \'hers\', \'herself\', \'it\', "it\'s", \'its\', \'itself\', \'they\', \'them\', \'their\',            \'theirs\', \'themselves\', \'what\', \'which\', \'who\', \'whom\', \'this\', \'that\', "that\'ll", \'these\', \'those\',             \'am\', \'is\', \'are\', \'was\', \'were\', \'be\', \'been\', \'being\', \'have\', \'has\', \'had\', \'having\', \'do\', \'does\',             \'did\', \'doing\', \'a\', \'an\', \'the\', \'and\', \'but\', \'if\', \'or\', \'because\', \'as\', \'until\', \'while\', \'of\',             \'at\',

In [ ]:
%%writefile app.py
import streamlit as st
import tqdm
#import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
#data = pd.read_csv('chrome_reviews.csv')
#data.dropna(inplace = True)

#def rating(s):

  #if(s>=3):
   # return 1
  #else:
    #return 0
#a = data['Star']
#b = a.map(rating)
#data['Star'] = b
stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])
pickle_in = open('Navie_model.pkl','rb')
classifier = pickle.load(pickle_in)

pickle_cv = open('cv-transform.pkl','rb')
cv = pickle.load(pickle_cv)
import re
def decontracted(text):

  text = re.sub(r"won't","will not",text)
  text = re.sub(r"can't","can not",text)
  text = re.sub(r"\'s","is",text)
  text = re.sub(r"\'re","are",text)
  text = re.sub(r"\'not","not",text)
  text = re.sub(r"\'ve","have",text)
  text = re.sub(r"\'d","would",text)
  text = re.sub(r"\'ll","will",text)
  text = re.sub(r"\'m","am",text)
  return text

def prediction(text):
  preprocessed_query = []
  text = decontracted(text)
  text = re.sub("\S*\d\S*","",text).strip() # remove words with numbers in python
  text = re.sub('[^A-Za-z0-9]+',' ',text)#remove special characters
  text = ' '.join(e.lower() for e in text.split() if e.lower() not in stopwords)
  preprocessed_query.append(text.strip())
  vect = cv.transform(preprocessed_query).toarray()
  print(vect)
  predictions = classifier.predict(vect)
  if(predictions==1):
    pred = 1
  else:
    pred = 0
  return pred


def main():

  #for front web page
  html_temp = """
  <div style ="background-color:yellow;padding:13px">
  <h1 style = "color:black;text-align:center;">Application for Mobile review
  </h1>
  </div>
  """
  st.markdown(html_temp,unsafe_allow_html=True)

  review = st.text_input("enter the input")

  result = " "

  if st.button("Predict"):
    result = prediction(review)
    st.success(result)

if __name__ == '__main__':
  main() 



Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!ngrok authtoken 24QK9tsxhgVhAIssPQhPrFNQSWg_3j6PtXWiKHCUnCtXn6BbG

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#!nohup streamlit run app.py

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect('8501')
public_url

2022-03-13 15:05:30.250 INFO    pyngrok.ngrok: Opening tunnel named: http-8501-52cb2a2a-3436-4bc3-b40c-523e920a3d1a
2022-03-13 15:05:30.255 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg=start pg=/api/tunnels id=446cfe5dac870a35
2022-03-13 15:05:30.651 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg="started tunnel" obj=tunnels name="http-8501-52cb2a2a-3436-4bc3-b40c-523e920a3d1a (http)" addr=http://localhost:8501 url=http://055b-34-80-155-237.ngrok.io
2022-03-13 15:05:30.654 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8501-52cb2a2a-3436-4bc3-b40c-523e920a3d1a addr=http://localhost:8501 url=https://055b-34-80-155-237.ngrok.io
2022-03-13 15:05:30.663 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg=end pg=/api/tunnels id=446cfe5dac870a35 status=201 dur=395.943772ms


<NgrokTunnel: "http://055b-34-80-155-237.ngrok.io" -> "http://localhost:8501">

2022-03-13 15:05:30.665 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg=start pg="/api/tunnels/http-8501-52cb2a2a-3436-4bc3-b40c-523e920a3d1a (http)" id=441e4f887f64d729
2022-03-13 15:05:30.669 INFO    pyngrok.process.ngrok: t=2022-03-13T15:05:30+0000 lvl=info msg=end pg="/api/tunnels/http-8501-52cb2a2a-3436-4bc3-b40c-523e920a3d1a (http)" id=441e4f887f64d729 status=200 dur=139.036µs
